
### Mounting the data volume

Start by creating a snapshot from the Pink Berries Metagenome snapshot, and starting a CeruleanTools AMI Instance with that volume loaded onto it.

You have to make a directory for the drive that contains the snapshot, and mount the drive to that directory. My metagenome data volume was at `/dev/xvdf`; I'm not sure how to tell where it will be before you load the volume onto the instance.

In [ ]:
!lsblk
!mkdir ~/data
!sudo mount /dev/xvdf ~/data

Alternatively, if you make a snapshot in the right zone (e.g. us-east-1d), you can load and mount the volume directly from your instance instead of selecting the snapshot at instance startup:

```
mkdir ~/data

aws ec2 attach-volume --volume-id vol-0bdfad3677d717075 --instance-id i-0a62227ff1d1977bd --device /dev/xvdh

sudo mount /dev/xvdh ~/data
```

### Grabbing binned data

Make a new directory where we'll do our work. Then, copy the Illumina short-read files in our new directory. Binning found both alphaproteobacteria ("a" prefix) and bacteroidetes ("b" prefix)--we only want to copy over the bacteroidetes.

We will use the "illumina_4moleculo" files; these were prepped for Moleculo sequencing. Illumina was also performed on PacBio-prepped sequences, but the coverage was not as high.

In [ ]:
!cd
!mkdir hybrid
!cp -R ~/data/metagenomes/sequence_reads/illumina_4moleculo/quality-trimmed-reads/reads-by-genome/b* ~/hybrid/reads-by-genome/

# II. Pre-processing

### Deinterleaving

These are interleaved files. To deinterleave each of them, run the following:
```
cd ~/hybrid/reads-by-genome/
mkdir ~/hybrid/deinterleaved
for FILE in *; do 
mkdir ~/hybrid/deinterleaved/$FILE-deinterleaved;
grep -A1 "_1$" "$FILE" | grep -v "^--$" >  ~/hybrid/deinterleaved/$FILE-deinterleaved/reads-1.fasta; 
grep -A1 "_2$" "$FILE" | grep -v "^--$" >  ~/hybrid/deinterleaved/$FILE-deinterleaved/reads-2.fasta; 
done
```

Depending on what the files are named, it might be good to tweak the above to give more sensible names to your folders. I did this manually. However, you probably want to keep the name of each pair of files within each folder the same: reads-1 and reads-2. This makes the assembly step simpler.

### Renaming reads 

ABySS also needs each read to be named with a slash. While in the directory containing the folders for each binned OTU (i.e. ~/hybrid/deinterleaved), the following will replace the

    hyphens (e.g. DJB775P1:392:D1R59ACXX:2:1310:17052:38927-2)
    with slashes (--> DJB775P1:392:D1R59ACXX:2:1310:17052:38927/2)


The -i flag is required to write the results of sed to a file (.bak is necessary for compatibility with certain systems). 

In [ ]:
!cd ~/hybrid/deinterleav d/
!for FOLDER in * ; do for FILE in $FOLDER/*; do sed -i.bak 's/_2/\/2/' $FILE; sed -i.bak 's/_1/\/1/' $FILE; done; done

This command creates some extraneous .bak files. Delete them:

In [ ]:
!cd ~/hybrid/deinterleaved/
!for FOLDER in * ; do rm $FOLDER/*.bak; done

### Reverse-complementing (binned reads only)

Due to the binning process, the binned reads are in a forward-forward read format, i.e. both paired-end reads are both from 5' to 3'. (For more conceptual information on this, see http://www.cureffi.org/2012/12/19/forward-and-reverse-reads-in-paired-end-sequencing/.) 

However, ABySS needs them to be in forward-reverse format, i.e. for each paired read, one needs to be reverse-complemented. We'll use Biopython to reverse-complement the reads.

Install Biopython using pip:

    pip install biopython

Then run the following python script in each binned folder to reverse-complement the `reads-2` files. Any lines that say "print" can be commented out if desired.

In [ ]:
#Reverse-complementing the reads in a fasta file, reads-2.fasta
#To be run within the folder in which each reads-2 file is located 

from Bio.Seq import Seq
from Bio import SeqIO

rc_file = open("rc-reads-2.fasta", "w+") #w opens file for writing; 
                                         #+ creates if it doesn't exist

for seq_record in SeqIO.parse("reads-2.fasta", "fasta"):
    print("Reverse-complementing: "+seq_record.id)
    #print("ORIGINAL: " + seq_record.seq)
    seqRC = seq_record.reverse_complement(id=True) #preserves seq ID
    #print("REV-COMP: " + seqRC.seq)
    print("Reverse-complementing complete! ") #+ seqRC.id + "\n")
    
    #write new record to file
    rc_file.write(">"+str(seqRC.id))
    rc_file.write("\n")
    rc_file.write(str(seqRC.seq))
    rc_file.write("\n")

rc_file.close()


# III. ABySS: assemble short-read contigs

First, we'll use ABySS to assemble the binned bacteroidetes paired-end contigs.

Now, assemble the contigs. The flag k=64 is the minimum k-mer length; any sequence shorter than this is discarded. It's probably a good idea to run this in a screen--or a series of screens for each bin. To check on what processes are running, use `top` (and `q` to quit)

In the binned case, I preferred doing this individually for each (reverse-complemented) bacteroidetes bin so I could specify a different name for each file, e.g.:

```
cd ~/hybrid/rev-comp/b1_flavo_deinterleaved
abyss-pe name=b1-flavo k=64 in='reads-1.fasta rc-reads-2.fasta'
```
and so on.

```
e.g.
abyss-pe name=b2-owen k=64 in='reads-1.fasta rc-reads-2.fasta'
abyss-pe name=b3-bact k=64 in='reads-1.fasta rc-reads-2.fasta'
abyss-pe name=b4-cyt1 k=64 in='reads-1.fasta rc-reads-2.fasta'
abyss-pe name=b5-cyt2 k=64 in='reads-1.fasta rc-reads-2.fasta'

```

For each run this generates 2 files used as inputs to Cerulean:
```
* <dataname>-contigs.fa    #This contains the contig sequences
* <dataname>-contigs.dot   #This contains the graph structure
```

# IV. BLASR: map ABySS contigs to long reads

The PacBio SmrtTools include BLASR, the program we will use next. The path for the PacBio tools ($SMRT_ROOT) was already set up in the CeruleanTools AMI.

### Setting up directories

Add a copy of the long reads (rename them to something nice, like pacbio.fasta) and the bacteroidetes *-contigs.fa files to a directory:

In [ ]:
!mkdir ~/hybrid/blasr
!cp ~/hybrid/rev-comp/*/*contigs.fa ~/hybrid/blasr/
!cp ~/data/metagenomes/sequence_reads/pacbio/corrected.fasta ~/hybrid/blasr/pacbio.fasta

### Running BLASR
We will use sawriter and BLASR, programs included in the Pacific Biosciences SMRTAnalyis toolkit that is installed on the CeruleanTools AMI.
   
Our PacBio reads are stored in `pacbio.fasta` and our contigs are stored in `b*-*-contigs.fa`.

Note that the `-nproc` flag is set to parallelize BLASR in 31 threads--this is only applicable if your machine has that many cores! 

```
 sawriter b1-flavo-contigs.fa
 blasr pacbio.fasta b1-flavo-contigs.fa -minMatch 10 \
     -minPctIdentity 70 -bestn 30 -nCandidates 30 -maxScore -500 \
     -nproc 31 -noSplitSubreads -header\
     -out b1-flavo_pacbio_contigs_mapping.fasta.m4
```

blasr pacbio.fasta b3-bact-contigs.fa -minMatch 10 \
     -minPctIdentity 70 -bestn 30 -nCandidates 30 -maxScore -500 \
     -nproc 31 -noSplitSubreads -header\
     -out b3-bact_pacbio_contigs_mapping.fasta.m4

#### IMPORTANT NOTE:
The -header flag lets us ensure that the fasta.m4 file generated has the following format:
```
qName tName qStrand tStrand score percentSimilarity tStart tEnd tLength qStart qEnd qLength nCells
```

However, this header line must be removed from the file before Cerulean is run! Alternatively, you can choose not to include the -header flag. The output of this command is a fasta.m4 file, which we will use in the next step.

# V. Cerulean: create assembly from BLASR map

Cerulean requires that all input files are in the same directory `<basedir>`. Note that our data are named such that `<dataname>` indicates the name of the bin (i.e. b1-flavo, b2-owen, ..., b5-cyt2). Thus we should have: 

1. `<basedir>/<dataname>-contigs.fa`
2. `<basedir>/<dataname>-contigs.dot`
3. `<basedir>/<dataname>_pacbio_contigs_mapping.fasta.m4`

Cerulean is run with the following format:
```
python src/Cerulean.py --dataname <dataname> --basedir <basedir> \
 --nproc <numthreads>
```
e.g.
```
python ~/Cerulean/src/Cerulean.py --dataname b1-flavo --basedir ~/hybrid/Cerulean \
 --nproc 31
```
 
This will generate:
1.  `<basedir>_cerulean.fasta`
2. `<basedir>_cerulean.dot`

Note: The .dot file does not have same contigs as fasta, but intermediate graph.

# VI. PBJelly: fill gaps in Cerulean assembly

Cerulean does not include consensus PacBio reads in gaps: the contigs are mapped, but there's just space in the gaps between them. These gaps may be filled using PBJelly. 

### Creating necessary .qual files
Cerulean produces a fasta file. But to use PBJelly, we need either a fastq file or a .qual file. Luckily, PBJelly provides us with fakeQuals.py, which generates a fake .qual file for us to use. (Note: in the CeruleanTools AMI, PBJelly's path is SWEETPATH.) We'll also use fakeQuals.py to make a .qual file for our fasta file, despite the fact that we do have a fastq file. 

(Would be worth checking to see if we can just use the original fastq file.)

Remember that <dataname> must be changed to whatever the filename is.
```
python $SWEETPATH/bin/fakeQuals.py <dataname>_cerulean.fasta <dataname>_cerulean.qual
python $SWEETPATH/bin/fakeQuals.py pacbio.fasta <dataname>_pacbio.qualm
```

### Modifying PBJelly Protocol.xml file:
We'll also copy over our PBJelly Protocol.xml file and make a new directory, PBJelly.

```
cp $SWEETPATH/docs/jellyExample/Protocol.xml <basedir>
mkdir PBJelly
```

Modify Protocol.xml as follows:

* Set `<reference>` to `$PATH_TO_<basedir>/<dataname>_cerulean.fasta`
* Set `<outputDir>` to `$PATH_TO_<basedir>/PBJelly`
* Set `<baseDir>` to `$PATH_TO_<basedir>`
* Set `<job>` to `pacbio.fasta`
* Set `<blasr>` option `-nproc <numthreads>`
 
(Note: PBJelly requires that the suffix be .fasta and not .fa)

### Running PBJelly:
 
Run PBJelly from within `<basedir>` in each of the following stages.

```
python $SWEETPATH/pbsuite/jelly/Jelly.py <stage> Protocol.xml
```
 
where <stage> has to be in the order:

```
setup
mapping
support
extraction
assembly
output
```
 
The assembled contigs are found in  
```
<basedir>/PBJelly/jelly.out.fasta
```
 
### KNOWN ERROR: "NO GAPS TO FILL"

# VII. Wishlist

1. Quality testing against other assemblies
2. Attempting to use fastq files instead of fasta files --> create fastq file and bypass PBJelly's fakeQuals.py
3. Going through similar process with Moleculo long reads instead of PacBio Long reads. Moleculo reads might be higher-quality, which would especially make a difference in gap-filling.